<a href="https://colab.research.google.com/github/erdenebayrd/llm/blob/main/phi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

LlamaTokenizerFast(name_or_path='microsoft/Phi-3-mini-4k-instruct', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=False),
	32000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<|placeholder1|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=

In [3]:
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", device_map="cuda", torch_dtype="auto", trust_remote_code=True)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
# Create a pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False, max_new_tokens=50, do_sample=False)

In [5]:
prompt = "The capital of France is"

#tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
input_ids.shape

torch.Size([1, 5])

In [6]:
# Get the output of the model before the lm_head
model_output = model.model(input_ids)
model_output[0].shape

torch.Size([1, 5, 3072])

In [37]:
model_output[0][0]

tensor([[-0.3047,  1.1953,  0.2988,  ..., -0.3008,  0.6758,  0.1406],
        [-0.1338,  0.3184,  0.3926,  ...,  0.5508, -0.1318, -0.6055],
        [-0.5820,  1.0703,  1.5547,  ..., -0.4082,  0.2773,  0.4062],
        [-0.3984,  0.6719,  0.2539,  ..., -0.0732,  0.8164, -0.6875],
        [-0.6758,  0.7070,  0.5469,  ...,  0.2598,  0.1621, -0.2773]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SelectBackward0>)

In [7]:
# Get the output of the lm_head
lm_head_output = model.lm_head(model_output[0])
lm_head_output.shape

torch.Size([1, 5, 32064])

In [40]:
lm_head_output[0]

tensor([[24.7500, 24.8750, 22.7500,  ..., 19.0000, 19.0000, 19.0000],
        [31.1250, 31.5000, 26.0000,  ..., 26.0000, 26.0000, 26.0000],
        [31.5000, 28.8750, 31.1250,  ..., 26.2500, 26.2500, 26.2500],
        [33.0000, 31.8750, 36.0000,  ..., 27.8750, 27.8750, 27.8750],
        [27.8750, 29.5000, 28.1250,  ..., 20.5000, 20.5000, 20.5000]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SelectBackward0>)

In [44]:
lm_head_output[0, 0]

tensor([24.7500, 24.8750, 22.7500,  ..., 19.0000, 19.0000, 19.0000],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SelectBackward0>)

In [62]:
token_id = lm_head_output[0, -1].argmax(-1)
token_id

tensor(3681, device='cuda:0')

In [63]:
tokenizer.decode(token_id)

'Paris'

In [65]:
# Assuming input_ids contains the prompt "The capital of France is"
# Use only the last token in the sequence for predicting the next token
last_token_hidden = model_output[0][:, -1, :]  # Shape: (batch_size, hidden_size)
last_token_hidden

tensor([[-0.6758,  0.7070,  0.5469,  ...,  0.2598,  0.1621, -0.2773]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SliceBackward0>)

In [66]:
next_token_logits = model.lm_head(last_token_hidden)  # Shape: (batch_size, vocab_size)
next_token_logits

tensor([[27.8750, 29.5000, 28.1250,  ..., 20.5000, 20.5000, 20.5000]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<MmBackward0>)

In [67]:
only_next_token_id = next_token_logits.argmax(-1)
only_next_token_id

tensor([3681], device='cuda:0')

In [68]:
tokenizer.decode(only_next_token_id)

'Paris'